In [19]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.insert(1,'/Users/jackh/Documents/FMBV_2023/gordon_original/') # FMBV src
# sys.path.insert(1,'/Users/jackh/Documents/FMBV_2023/gordon_original/src/legacy_src/')

import glob, os
import pandas as pd

# import fmbv_functions as fmbv
import load 

import time
from fmbv_ import FMBV
import fmbv_refactor

import warnings
warnings.filterwarnings('ignore')

# batch_path = "C:/Users/z5165170/Documents/Perfusion/Data/wl1/"
batch_path = "../../test_batch/"
kretz_filepaths = glob.glob(batch_path+"**.vol")
# print(kretz_filepaths)
# output_df = pd.DataFrame(columns=['Name','FMBV Original', 'FMBV Refactor', 'FMBV Split', 'FMBV Modified', 'FMBV DC Original', 'FMBV DC Refactor', 'FMBV DC Split', 'FMBV DC Modified'])
data = []

for i in range(len(kretz_filepaths))[0:]:
    print("Analysing "+str(i)+"...")

    kretz_path = kretz_filepaths[i]
    # print(kretz_path)
    i_noext = os.path.splitext(kretz_path)[0]
    pd_path = i_noext+'_dp.nii.gz'
    seg_path = i_noext+'_seg.nii.gz'
    print(kretz_path)

    max_pixel_value = 255.0

    # Original Gordon Call
    df_gordon = load.run_paths(
        pd_path = pd_path,
        seg_path = seg_path,
        kretz_path = kretz_path
    )

    # Mode 0
    f0 = fmbv_refactor.FMBV(
        verbose=False,
        pd_low_threshold=2.0,
        max_pixel_value = max_pixel_value,
        clip = True,
        mode = 0
        )
    f0.load_pd(pd_path)
    f0.load_seg(seg_path)
    f0.load_kretz(kretz_path)
    f0.depth_corrected_method()
    f0.global_method()

    # Mode 1
    f1 = fmbv_refactor.FMBV(
        verbose=False,
        pd_low_threshold=2.0,
        max_pixel_value = max_pixel_value,
        clip = True,
        mode = 1
        )
    f1.load_pd(pd_path)
    f1.load_seg(seg_path)
    f1.load_kretz(kretz_path)
    f1.depth_corrected_method()
    f1.global_method()

    # Mode 1 via Front-End
    df_1 = load.run_paths(
        pd_path = pd_path,
        seg_path = seg_path,
        kretz_path = kretz_path,
        max_pixel_value = max_pixel_value,
        mode = 2
    )

    data.append([df_gordon.at[0,'FMBV'], f0.global_fmbv_value_2, f1.global_fmbv_value_2, df_1.at[0,'FMBV'],
                 df_gordon.at[0,'FMBV Depth Corrected'], f0.dc_fmbv, f1.dc_fmbv, df_1.at[0,'FMBV Depth Corrected']])

Analysing 0...
../../test_batch/wl1_8.vol
Analysing 1...
../../test_batch/wl1_1.vol
Analysing 2...
../../test_batch/wl1_10.vol
FMBV Failed at Step 1!ds for axis 0 with size 3index 5 is out of bounds for axis 0 with size 5
index 5 is out of bounds for axis 0 with size 5
index 3 is out of bounds for axis 0 with size 3
Analysing 3...
../../test_batch/wl1_2.vol
Analysing 4...
../../test_batch/wl1_12.vol
FMBV Failed at Step 1!ds for axis 0 with size 3expected non-empty vector for x
expected non-empty vector for x
expected non-empty vector for x
index 5 is out of bounds for axis 0 with size 5
index 3 is out of bounds for axis 0 with size 3
Analysing 5...
../../test_batch/wl1_3.vol
Analysing 6...
../../test_batch/wl1_7.vol
FMBV Failed at Step 1!ds for axis 0 with size 5expected non-empty vector for x
index 5 is out of bounds for axis 0 with size 5
Analysing 7...
../../test_batch/wl1_6.vol
Analysing 8...
../../test_batch/wl1_4.vol
FMBV Failed at Step 1!ds for axis 0 with size 0index 0 is out o

In [21]:
df_output = pd.DataFrame(data)
print(df_output)

           0          1          2          3          4          5  \
0  32.750524  32.750524  25.669460  25.669460  38.198519  38.198519   
1  36.849886  36.849886  33.717114  33.717114  42.673519  42.673519   
2  23.241536  23.241536  17.641679  17.641679  29.101490  29.101490   
3  38.405993  38.405993  29.370082  29.370082  44.884061  44.884061   
4  15.600887  15.600887  12.167355  12.167355  20.024243  20.024243   
5  40.923845  40.923845  30.883135  30.883135  47.393281  47.393281   
6  18.898238  18.898238  15.625391  15.625391  27.445282  27.445282   
7  28.965154  28.965154  26.138447  26.138447  36.899636  36.899636   
8  33.164481  33.164481  31.345809  31.345809  38.593041  38.593041   
9  33.380597  33.380597  31.178340  31.178340  38.716983  38.716983   

           6          7  
0  35.320705  35.320705  
1  36.275651  36.275651  
2  24.273982  24.273982  
3  31.871389  31.871389  
4  19.042719  19.042719  
5  34.322146  34.322146  
6  21.574502  21.574502  
7  32.1832